In [ ]:
import pandas as pd
from sqlalchemy import create_engine,text
import pymysql.cursors
import os
# import getpass
import urllib.parse

In [ ]:
pw_raw = os.getenv('mysql_xt')
pw = urllib.parse.quote_plus(pw_raw)

In [ ]:
connection_string = 'mysql+pymysql://root:' + pw + '@127.0.0.1:3306/'
engine = create_engine(connection_string)

In [ ]:
with engine.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS project_ab_testing"))

In [ ]:
client_prof=pd.read_csv('df_final_demo.txt')
client_prof.to_sql('client_profile',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
experiment_roster=pd.read_csv('df_final_experiment_clients.txt')
experiment_roster.to_sql('experiment_roster',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
pt1=pd.read_csv('df_final_web_data_pt_1.txt')
pt2=pd.read_csv('df_final_web_data_pt_2.txt')
digital_footprints = pd.concat([pt1,pt2])
digital_footprints = digital_footprints[~digital_footprints.duplicated()]
digital_footprints.to_sql('digital_footprints',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
df_with_variation = digital_footprints.merge(experiment_roster, on='client_id', how='left').sort_values(by="date_time")
test_group = df_with_variation[df_with_variation['Variation'] == 'Test']
test_group.to_sql('test_group',engine, 'project_ab_testing', if_exists='replace', index=False)
control_group = df_with_variation[df_with_variation['Variation'] == 'Control']
control_group.to_sql('control_group',engine, 'project_ab_testing', if_exists='replace', index=False)